In [1]:
#import dependencies
import pandas as pd 
import matplotlib.pyplot as plt
import requests
import json
import scipy.stats as st
import sklearn.datasets as dta
import numpy as np 
from datetime import datetime
import random
import warnings
warnings.filterwarnings('ignore')

In [2]:
#call in csv file and set up API keys
microplastics_csv = pd.read_csv("Resources/MarinePlastics.csv")

microplastics_csv.head()
#test
#test2

,FID,Date,Latitude,Longitude,Oceans,Regions,SubRegions,Measurement,Unit,Density Range,...,Accession Number,Accession Link,GlobalID,CreationDate,Creator,EditDate,Editor,Long Reference,x,y
0,35217,4/6/2005 12:00:00 AM,2.19,-135.73,Pacific Ocean,NaN,NaN,0.00000,pieces/m3,0-0.0005,...,211008,https://www.ncei.noaa.gov/access/metadata/land...,6d278226-565e-4ed0-b46a-e1cabb32ecc0,3/9/2023 9:05:33 PM,ncei_cds_noaa,3/9/2023 9:05:33 PM,ncei_cds_noaa,"Law, K.L, S.K. Morét-Ferguson, D.S. Goodwin, E...",-1.510939e+07,2.438491e+05
1,36142,2/24/2002 12:00:00 AM,-12.69,-142.30,Pacific Ocean,NaN,NaN,0.00000,pieces/m3,0-0.0005,...,211008,https://www.ncei.noaa.gov/access/metadata/land...,6fb35a58-21b6-4cce-9dbb-f9c222bc311d,3/9/2023 9:05:33 PM,ncei_cds_noaa,3/9/2023 9:05:33 PM,ncei_cds_noaa,"Law, K.L, S.K. Morét-Ferguson, D.S. Goodwin, E...",-1.584076e+07,-1.424337e+06
2,35881,10/25/2010 12:00:00 AM,19.62,-146.50,Pacific Ocean,NaN,NaN,0.00216,pieces/m3,0.0005-0.005,...,211008,https://www.ncei.noaa.gov/access/metadata/land...,67f360ab-017e-4a86-931b-6f7336c2c206,3/9/2023 9:05:33 PM,ncei_cds_noaa,3/9/2023 9:05:33 PM,ncei_cds_noaa,"Law, K.L, S.K. Morét-Ferguson, D.S. Goodwin, E...",-1.630831e+07,2.228069e+06
3,36474,11/16/2003 12:00:00 AM,18.97,-143.47,Pacific Ocean,NaN,NaN,0.00000,pieces/m3,0-0.0005,...,211008,https://www.ncei.noaa.gov/access/metadata/land...,5be50d3b-7343-45f0-ac71-19a3cdc047ea,3/9/2023 9:05:33 PM,ncei_cds_noaa,3/9/2023 9:05:33 PM,ncei_cds_noaa,"Law, K.L, S.K. Morét-Ferguson, D.S. Goodwin, E...",-1.597101e+07,2.151404e+06
4,34553,3/5/2001 12:00:00 AM,19.38,-74.09,Atlantic Ocean,Caribbean Sea,NaN,0.00000,pieces/m3,0-0.0005,...,211007,https://www.ncei.noaa.gov/access/metadata/land...,5e54fac6-640b-4445-8fff-94a9498d6168,3/9/2023 9:05:33 PM,ncei_cds_noaa,3/9/2023 9:05:33 PM,ncei_cds_noaa,"Law, K.L., S. Morét-Ferguson, N.A. Maximenko, ...",-8.247661e+06,2.199726e+06


In [3]:
#taking columns we need
clean_microplastics_df = microplastics_csv[["Date","Oceans","Measurement","Unit","Sampling Method"]]
clean_microplastics_df["Year"] = pd.Series()
clean_microplastics_df.head()

,Date,Oceans,Measurement,Unit,Sampling Method,Year
0,4/6/2005 12:00:00 AM,Pacific Ocean,0.00000,pieces/m3,Neuston net,NaN
1,2/24/2002 12:00:00 AM,Pacific Ocean,0.00000,pieces/m3,Neuston net,NaN
2,10/25/2010 12:00:00 AM,Pacific Ocean,0.00216,pieces/m3,Neuston net,NaN
3,11/16/2003 12:00:00 AM,Pacific Ocean,0.00000,pieces/m3,Neuston net,NaN
4,3/5/2001 12:00:00 AM,Atlantic Ocean,0.00000,pieces/m3,Neuston net,NaN


In [4]:
#cleaning time column
for i in range(len(clean_microplastics_df["Date"])):
    date_time = clean_microplastics_df.loc[i,"Date"]
    date = date_time.split(" ")[0]
    clean_microplastics_df.loc[i,"Date"]=date
clean_microplastics_df.head()

,Date,Oceans,Measurement,Unit,Sampling Method,Year
0,4/6/2005,Pacific Ocean,0.00000,pieces/m3,Neuston net,NaN
1,2/24/2002,Pacific Ocean,0.00000,pieces/m3,Neuston net,NaN
2,10/25/2010,Pacific Ocean,0.00216,pieces/m3,Neuston net,NaN
3,11/16/2003,Pacific Ocean,0.00000,pieces/m3,Neuston net,NaN
4,3/5/2001,Atlantic Ocean,0.00000,pieces/m3,Neuston net,NaN


In [5]:
for x in range(len(clean_microplastics_df["Year"])):
    year_time = clean_microplastics_df.loc[x,"Date"]
    year = year_time.split("/")[2]
    clean_microplastics_df.loc[x,"Year"]=year
clean_microplastics_df.tail()
    

,Date,Oceans,Measurement,Unit,Sampling Method,Year
11301,11/28/2014,Atlantic Ocean,0.00200,pieces/m3,Grab sample,2014
11302,12/11/2014,Atlantic Ocean,0.02500,pieces/m3,Grab sample,2014
11303,11/7/1995,Atlantic Ocean,0.00864,pieces/m3,Neuston net,1995
11304,10/31/2001,Atlantic Ocean,0.01080,pieces/m3,Neuston net,2001
11305,5/21/2014,Atlantic Ocean,0.00200,pieces/m3,Grab sample,2014


In [26]:
# sampling_mean = clean_microplastics_df.groupby(["Sampling Method"]).mean()["Measurement"]
# sampling_median = clean_microplastics_df.groupby(["Sampling Method"]).median()["Measurement"]
# sampling_max = clean_microplastics_df.groupby(["Sampling Method"]).max()["Measurement"]
# sampling_min = clean_microplastics_df.groupby(["Sampling Method"]).min()["Measurement"]

# sampling_statistics = pd.DataFrame ({
#     "Mean Measument per Sampling": sampling_mean,
#     "Median Measument per Sampling": sampling_median,
#     "Max Measument per Sampling": sampling_max,
#     "Min Measument per Sampling": sampling_min})

# sampling_statistics.head()

,Mean Measument per Sampling,Median Measument per Sampling,Max Measument per Sampling,Min Measument per Sampling
Sampling Method,,,,
Aluminum bucket,2089.859649,1710.000000,7650.333333,703.0
Grab sample,0.012880,0.005000,0.323000,0.0
Intake seawater,26.730769,27.000000,63.000000,0.0
Manta net,2.887066,1.083789,51.078200,0.0
Neuston net,0.082112,0.002160,49.267784,0.0


In [25]:
atlantic_mean = clean_microplastics_df[clean_microplastics_df["Oceans"] == "Atlantic Ocean"].groupby("Year")["Measurement"].mean()

atlantic_mean_df = pd.DataFrame({
    "Year": atlantic_mean.index,
    "Average microplastics Per Year in the Pacific Ocean": atlantic_mean.values
})

atlantic_mean_df

,Year,Average microplastics Per Year in the Pacific Ocean
0,1986,0.027291
1,1987,0.024966
2,1989,0.024941
3,1990,0.025754
4,1991,0.019063
5,1992,0.017570
6,1993,0.020521
7,1994,0.011186
8,1995,0.027987
9,1996,0.040504


In [22]:
#finding the mean value per year of microplastics in the Pacific ocean
pacific_mean = clean_microplastics_df[clean_microplastics_df["Oceans"] == "Pacific Ocean"].groupby("Year")["Measurement"].mean()

pacific_mean_df = pd.DataFrame({
    "Year": pacific_mean.index,
    "Average microplastics Per Year in the Pacific Ocean": pacific_mean.values
})

pacific_mean_df

,Year,Average microplastics Per Year in the Pacific Ocean
0,1972,0.016419
1,1973,0.022286
2,1987,0.014000
3,1999,0.010800
4,2000,0.026167
5,2001,0.002437
6,2002,0.008121
7,2003,0.009513
8,2004,0.057162
9,2005,0.075480


In [27]:
arctic_mean = clean_microplastics_df[clean_microplastics_df["Oceans"] == "Arctic Ocean"].groupby("Year")["Measurement"].mean()

arctic_mean_df = pd.DataFrame({
    "Year": arctic_mean.index,
    "Average microplastics Per Year in the Arctic Ocean": arctic_mean.values
})

arctic_mean_df

,Year,Average microplastics Per Year in the Arctic Ocean
0,2014,0.014000
1,2015,0.021000
2,2016,0.032905
3,2021,0.036970


In [9]:
date_mean = clean_microplastics_df.groupby(["Year"]).mean()["Measurement"]
date_mean_df = pd.DataFrame({
    "Average microplastics found Per Year": date_mean})
date_mean_df

,Average microplastics found Per Year
Year,
1972,0.016419
1973,0.022286
1986,0.027291
1987,0.023399
1989,0.024941
1990,0.025754
1991,0.019063
1992,0.017570
1993,0.020521


In [ ]:
#creating a box plot for all the oceans
ocean_list = ["Pacific","Atlantic","Arctic"]
plt.boxplot

In [10]:
# #display results for microplastics over time

# sample_df = clean_microplastics_df.sample(n=1000)
# sample_df.head()

# x_values = sample_df["Year"]
# y_values = sample_df["Measurement"]
# plt.ylim(0,1)
# plt.bar(x_values,y_values)
# plt.show()


In [11]:
# sample_df.boxplot(column="Measurement",by="Date")
# plt.show()

In [12]:
#display results for cancer rates over time 

In [13]:
#display regression plots 